Coding Assignment
1.	Download video file (airshow.mp4) from https://www.dropbox.com/sh/782f7a0hmw3mn59/AADcJpRoFB0IIfi7-nR-7Eifa?dl=0.

2.	Cut it up into 1-minute clips, save as .mov or .mp4 inside a folder named “video_clips”. (Depending on the method you use, it may not be exactly 1-min. Roughly 1-min is fine.) Please create folder programmatically with Python.

3.	The names of each clip must include the starting frame count of the cut video. For example, the 1st clip may be named 0thFrame.mov, the 2nd clip may be named 1800thFrame.mov, etc. Depending on how you chop up the video, the frame with which you start the next clip may not be exactly 1800 frames after. That is fine as long as that is correctly reflected on the saved name of the clip.

4.	Create table video_data in PostgresSQL database with the following columns:
a.	clip_name (name of the video clip created in section 2 above, excluding file extension);
b.	clip_file_extension (file extension of the video clip created in section 2 above);
c.	clip_duration (duration of the video clip created in section 2 above);
d.	clip_location (location of the video clip created in section 2 above. It can be local path to the project directory or the absolute path);
e.	insert_timestamp (a timestamp when a record has been inserted into a table).
You have the freedom to determine the field types. This video_data table must be created programmatically with Python. Don’t use any SQL tools to create this table.
5.	After creating video_data table insert all the records with information described in section 4 above. It should be 1 record per 1 generated clip.

6.	The same records that have been inserted into the video_data table must be saved to a CSV file named generated_video_files under “report” folder. Please create folder programmatically with Python.

Feel free to utilize any Python packages to solve this. 
Upload your code and a report (CSV file) from section 6 above to your Github account if possible and add the README.md file to the repository with instructions on how to setup the environment, run your code and any other notes you find important to mention. The repository does not have to contain the output video clips generated in section 2 above.


In [3]:
#Instalación de libreria de manipulación de videos
pip install moviepy

In [8]:
# Crear la carpeta en la cual estarán los videos
import os 
os.makedirs("video_clips")

In [14]:
from moviepy.editor import VideoFileClip

#Seleccionar el video
video = VideoFileClip("airshow.mp4") 

duration = video.duration 

#Sacar los frames por segundo para hacer calculos
frame_rate = video.fps 

for i in range(0, int(duration), 60): 
  start = i 
  end = min(i + 60, duration) 
    #Crear los cortes
  clip = video.subclip(start, end) 
    #Calcular el frames de inicio
  frame_count = int(start * frame_rate) 
    #Dar nombre al video nuevo
  file_name = f"{frame_count}thFrame.mp4" 
  file_path = os.path.join("video_clips", file_name)
    #Escribir el video nuevo 
  clip.write_videofile(file_path) 


Moviepy - Building video video_clips\0thFrame.mp4.
MoviePy - Writing audio in 0thFrameTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\0thFrame.mp4



Moviepy - Done !
Moviepy - video ready video_clips\0thFrame.mp4
Moviepy - Building video video_clips\1798thFrame.mp4.
MoviePy - Writing audio in 1798thFrameTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\1798thFrame.mp4



Moviepy - Done !
Moviepy - video ready video_clips\1798thFrame.mp4
Moviepy - Building video video_clips\3596thFrame.mp4.
MoviePy - Writing audio in 3596thFrameTEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready video_clips\5394thFrame.mp4
Moviepy - Building video video_clips\7192thFrame.mp4.
MoviePy - Writing audio in 7192thFrameTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\7192thFrame.mp4



Moviepy - Done !
Moviepy - video ready video_clips\7192thFrame.mp4
Moviepy - Building video video_clips\8991thFrame.mp4.
MoviePy - Writing audio in 8991thFrameTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\8991thFrame.mp4



Moviepy - Done !
Moviepy - video ready video_clips\8991thFrame.mp4


In [3]:
import sqlite3

#Crear folder nuevo
os.mkdir("report")


#Crear una conexión conn a la base de datos "video.db"
conn = sqlite3.connect("video.db")

#Crear un cursor
cur = conn.cursor()

#Se crea la tabla ("real" es igual a float)
cur.execute("""
    CREATE TABLE video_data (
        clip_name TEXT,
        clip_file_extension TEXT,
        clip_duration REAL, 
        clip_location TEXT,
        insert_timestamp DATETIME
    )
""")


In [15]:
import datetime
#Crear una lista nueva para almacenar los datos
records = []

#Se hace un ciclo for para leer todos los videos y sus datos
for file in os.listdir("video_clips"):
    #Nombre
    clip_name = os.path.splitext(file)[0]
    #Extension
    clip_file_extension = os.path.splitext(file)[1]
    #Duracion
    clip_duration = VideoFileClip(f"video_clips/{file}").duration
    #Path
    clip_location = os.path.join("video_clips", file)
    #Timestamp
    now = datetime.datetime.now()
    insert_timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
   
    records.append((clip_name, clip_file_extension, clip_duration, clip_location, insert_timestamp))

#Ingresar los datos
cur.executemany("""
    INSERT INTO video_data (clip_name, clip_file_extension, clip_duration, clip_location, insert_timestamp)
    VALUES (?, ?, ?, ?, ?)
""", records)

#Commit y terminar conexión
conn.commit()
conn.close()

In [16]:
import pandas as pd

#Conectar al archivo de la base de datos
conn = sqlite3.connect("video.db")

#Jalar los datos de la base de datos al data frame
df = pd.read_sql_query("SELECT * FROM video_data", conn)

#Generar el reporte
df.to_csv("report/baseDeDatos.csv", index=False)

#Se cierra la conexión
conn.close()